<div align="right"><i>Peter Norvig<br>Sept 25, 2024<br>Update May 21, 2025</i></div>

# LLMs, Theory of Mind, and Cheryl's Birthday

There has been [much](https://spectrum.ieee.org/theory-of-mind-ai) [debate](https://aclanthology.org/2023.conll-1.25/) [on](https://www.gsb.stanford.edu/faculty-research/working-papers/theory-mind-may-have-spontaneously-emerged-large-language-models) [the](https://arxiv.org/abs/2302.02083) [degree](https://www.nature.com/articles/s41562-024-01882-z) to which Large Language Models (LLMs) have a [theory of mind](https://en.wikipedia.org/wiki/Theory_of_mind): a way of understanding what other people know and don't know. In this notebook I explore one small part of the issue by asking nine LLMs to solve the [Cheryl's Birthday Problem](https://en.wikipedia.org/wiki/Cheryl%27s_Birthday), a well-known logic puzzle in which different characters have different states of knowledge at different times. The LLMs were all familiar with the puzzle, so if I just asked them to solve it, they all would just recall the answer they have memorized, "July 16." Instead  I test the LLMs  with the following two prompts, one a novel formulation that alters the dates and wording, and one that requires writing a general program that works for any set of dates:

## Reasoning Test

1. **Ali and Bo are friends with Cam. Cam told them that her anniversary is one of 10 possible dates:**
   - **April 17, April 18, April 28, July 16, July 17, July 19, June 16, June 29, March 18, March 19**
3. **Cam then privately tells Ali the month and Bo the day number of the anniversary.**
4. **Ali: "I don't know when Cam’s anniversary is, and I know that Bo does not know it either."**
5. **Bo: "At first I didn't know when Cam’s anniversary was, but I know now, after Ali's statement."**
6. **Ali: "Then I also know when Cam’s anniversary is."**
7. **When is Cam’s anniversary?**

## Programming Test

- **What is the answer to the "Cheryl's Birthday" problem? Write a Python program to solve it. Make sure that the program will still work if the list of possible dates is changed.**


## Results

The correct answer for the reasoning test is **March 18**, and for the programming test, **July 16**.

In **September 2024**, all of the LLMs failed both tests. They couldn't distinguish the different knowledge states of Alfred and Bernard. At least with respect to this problem, they had a poorly developed theory of mind. 

But in **May 2025**, all the LLMs had new versions, and three passed the programming test while four passed the reasoning test:

|2024<br>Model/version|2025<br> version|2024<br> Program|2024<br> Reasoning|2025<br> Program|2025<br> Reasoning|
|--|--|:--:|:--:|:-:|:--:|
|[A human](https://github.com/norvig/)|older human|✅|✅|✅|✅|
|[Gemini Advanced](https://gemini.google.com/app)|2.5 Pro|❌ |❌|✅ |✅|
|[You.com](https://you.com/)|Compute|❌ |❌|✅ |✅|
|[Anthropic Claude 3.5 Sonnet](https://claude.ai/new)|Sonnet 4|❌ |❌|✅ |❌  |
|[ChatGPT 4o](https://chatgpt.com/)|o4-mini-high|❌ |❌|❌ |✅|
|[Perplexity](https://www.perplexity.ai/)|Pro|❌ |❌|❌ |✅|
|[Microsoft Copilot](https://copilot.microsoft.com)|Copilot|❌|❌ |❌|❌|
|[Meta AI Llama 405B](https://www.meta.ai/)|AI|❌ |❌|❌ |❌|
|[Cohere Chat](https://cohere.com/chat)|Command A|❌ |❌|❌ |❌|
|[HuggingFace Chat](https://huggingface.co/chat/)|v0.9.4|❌|❌ |❌|❌|

# 2024 and 2025: Reasoning

As mentioned above, I created a reworded version of the puzzle and used it as a prompt:

1. **Ali and Bo are friends with Cam. Cam told them that her anniversary is one of 10 possible dates:**
   - **April 17, April 18, April 28, July 16, July 17, July 19, June 16, June 29, March 18, March 19**
3. **Cam then privately tells Ali the month and Bo the day number of the anniversary.**
4. **Ali: "I don't know when Cam’s anniversary is, and I know that Bo does not know it either."**
5. **Bo: "At first I didn't know when Cam’s anniversary was, but I know now, after Ali's statement."**
6. **Ali: "Then I also know when Cam’s anniversary is."**
7. **When is Cam’s anniversary?**

(The correct answer is **March 18**.)

In **2024**, all the LLMs were generally headed in the right direction in their reasoning, but all made mistakes. For example, Claude says "*Bo hears the day and realizes after Ali's statement. Since Bo did not initially know the date, the day number Bo heard must appear in more than one month. Therefore, the days 16, 18, and 19 must be eliminated since they have corresponding unique months.*" But that's just not right; they don't have unique months. As it turns out, [http://you.com](you.com) did get the right answer on the first try. But some of the reasoning steps were wrong, so I re-ran it, and it failed the second time. Thus, all the LLMs failed in 2024. 

In **2025**, Gemini, You.com, ChatGPT, and Perplexity got the right answer, with well-reasoned steps. The others made mistakes similar to the 2024 versions. I'll only show one example of reasoning, by Gemini 2.5 Pro:


## Gemini Pro 2.5 


This is a classic logic puzzle that can be solved by systematically eliminating possibilities based on the statements made by Ali and Bo.

1. The Initial Possibilities:

Let's list the dates by month:

- March: 18, 19
- April: 17, 18, 28
- June: 16, 29
- July: 16, 17, 19

2. Ali's First Statement: "I don't know when Cam’s anniversary is, and I know that Bo does not know it either."

"I don't know...": This tells us that the month Ali was told must have more than one possible date. All four months (March, April, June, July) have multiple possibilities, so this part doesn't eliminate anything yet.
"...and I know that Bo does not know it either.": This is the crucial part. If Ali knows Bo doesn't know, it means that no matter which day is possible in her month, that day number must also appear in another month. If there was a day number unique to her month, she couldn't be sure Bo didn't know.
Let's look for unique day numbers in the original list:

- 28 appears only in April.
- 29 appears only in June.

If Ali was told April, she couldn't be sure Bo didn't know, because if the date was April 28, Bo (having '28') would know immediately.
If Ali was told June, she couldn't be sure Bo didn't know, because if the date was June 29, Bo (having '29') would know immediately.
Therefore, Ali must have been told a month where all possible days also appear in other months. This eliminates April and June.

3. The Remaining Possibilities (after Ali's first statement):

- March: 18, 19
- July: 16, 17, 19

4. Bo's Statement: "At first I didn't know when Cam’s anniversary was, but I know now, after Ali's statement."

Bo now knows the date. This means that after eliminating April and June, his day number must now point to only one possible date among the remaining options (March 18, 19; July 16, 17, 19).
Let's check the day numbers in the remaining list:
- 16: Only appears in July 16. If Bo had 16, he would know.
- 17: Only appears in July 17. If Bo had 17, he would know.
- 18: Only appears in March 18. If Bo had 18, he would know.
- 19: Appears in March 19 and July 19. If Bo had 19, he would still not know.

Since Bo now knows, his day number cannot be 19. This eliminates March 19 and July 19.

5. The Remaining Possibilities (after Bo's statement):

- March: 18
- July: 16, 17

6. Ali's Second Statement: "Then I also know when Cam’s anniversary is."

Ali knows the month. She now knows the only possibilities are March 18, July 16, and July 17.
For her to know the exact date, her month must only have one option left in this list.
If her month was July, the options would be July 16 and July 17 – she wouldn't know.
If her month was March, the only option is March 18 – she would know.
Since Ali now knows, her month must be March.
Conclusion:

The only remaining possibility that fits all the statements is March 18. Therefore, Cam's anniversary is March 18.

# 2024: Programming

None of the LLMs produced a correct program in 2024. Each LLM provided explanatory output along with a program; for brevity I only show the explanatory output from the first LLM, ChatGPT 4o, but I'll show the code for all hte LLMs. My comments in *[bracketed italics]* explain where they went wrong.

## Human

An actual human programmer (me) was able to write a program, shown in [**another notebook**](https://github.com/norvig/pytudes/blob/main/ipynb/Cheryl-and-Eve.ipynb), that correctly solves the original problem and also handles different sets of dates. I introduced the idea of a *BeliefState*, a set of possible dates that a person believes might possibly be the birthday, and I modeled a character's *statement* as a function that takes a particular date as input, and returns true if the date is consistent with the statement.

## ChatGPT 4o

*[ChatGPT 4o gives the following response, which starts with an excellent summary of the problem:]*

The "Cheryl's Birthday" problem is a well-known logic puzzle that can be solved using logical deduction. Let's first break down the problem before writing a Python program.

### Problem Statement
Cheryl gives Albert and Bernard a list of 10 possible birthdates, and tells them different information:
- Albert knows the month.
- Bernard knows the day.

Cheryl provides the following 10 possible dates:
- May 15, May 16, May 19
- June 17, June 18
- July 14, July 16
- August 14, August 15, August 17

Albert then says: "I don't know when Cheryl's birthday is, but I know that Bernard doesn't know either."

Bernard replies: "At first I didn’t know when Cheryl's birthday is, but now I know."

Albert then says: "Now I know when Cheryl's birthday is."

### Steps for Deduction
1. From Albert's statement, we can deduce that Cheryl's birthday is not in May or June, because if it were, Bernard could possibly know the exact birthday if the day was unique (May 19 or June 18). Since Albert is sure Bernard doesn't know, we can eliminate May and June.
2. After eliminating May and June, the remaining possible dates are:
    - July 14, July 16
    - August 14, August 15, August 17
3. From Bernard’s statement, now knowing the birthday, we can deduce that Cheryl's birthday isn't on July 14 or August 14, because if it were, Bernard would still be uncertain (since both dates share the day 14).
4. The remaining possible dates are:
    - July 16
    - August 15
    - August 17
5. Finally, from Albert’s statement, we deduce that Cheryl's birthday must be in July, because if it were in August, Albert wouldn’t be able to know the exact date. Thus, the birthday is July 16.

### Python Program to Solve the Problem
We can now write a Python program to generalize this solution and ensure it works for any list of possible dates.

### Explanation:
- Step 1: The function groups the dates by month and by day.
- Step 2: Based on Albert's statement, we eliminate the months of May and June.
- Step 3: Based on Bernard's statement, we eliminate dates where the day appears more than once (i.e., both July 14 and August 14 are eliminated).
- Step 4: Albert now knows Cheryl's birthday, so it must be in the month where only one date is left (i.e., July).
- 
The program is designed to work with any given list of possible dates, not just the ones in the original problem.

In [1]:
def filter_dates(possible_dates):
    # Step 1: Group dates by month and day
    month_map = {}
    day_map = {}
    
    for date in possible_dates:
        month, day = date.split()
        day = int(day)
        
        # Group by month
        if month not in month_map:
            month_map[month] = []
        month_map[month].append(day)
        
        # Group by day
        if day not in day_map:
            day_map[day] = []
        day_map[day].append(month)
    
    # Step 2: Albert says: "I don't know when Cheryl's birthday is, but I know Bernard doesn't know either."
    # This implies Cheryl's birthday cannot be in May or June because these months have unique days (19th and 18th).
    possible_dates = [date for date in possible_dates if date.split()[0] not in ('May', 'June')]
    
    # Step 3: Bernard now knows the birthday, so it cannot be a day that appears more than once.
    possible_dates = [date for date in possible_dates if len(day_map[int(date.split()[1])]) == 1]
    
    # Step 4: Albert now knows the birthday, so it must be in a unique month remaining after Bernard's elimination.
    remaining_months = set(date.split()[0] for date in possible_dates)
    for date in possible_dates:
        month = date.split()[0]
        if len(month_map[month]) == 1:
            return date
    
    return "No solution found"

# Test the program with the given list of possible dates
possible_dates = [
    "May 15", "May 16", "May 19",
    "June 17", "June 18",
    "July 14", "July 16",
    "August 14", "August 15", "August 17"
]

cheryls_birthday = filter_dates(possible_dates)
print("Cheryl's birthday is:", cheryls_birthday)

Cheryl's birthday is: No solution found


*[We see that the program fails to find a solution. It starts with the correct list of "possible_dates", and correctly updates the list twice to reflect Albert's and Bernard's first statements. So far so good. But then things go wrong: first it defines "remaining_months" and  never uses it. Then it tries to find a possible date in a unique month, but it should be a unique month with respect to what is now known, not with respect to the original possible dates stored in the "month_map". The program has failed to track the two character's different belief states over time.]* 

## Microsoft Copilot

*[Copilot makes a similar mistake.*]

In [2]:
def cheryls_birthday(possible_dates):
    # Step 1: Filter out months with unique days
    unique_days = {date.split()[1] for date in possible_dates if sum(d.split()[1] == date.split()[1] 
                                   for d in possible_dates) == 1}
    possible_dates = [date for date in possible_dates if date.split()[1] not in unique_days]

    # Step 2: Filter out dates with unique months
    unique_months = {date.split()[0] for date in possible_dates if sum(d.split()[0] == date.split()[0] 
                                     for d in possible_dates) == 1}
    possible_dates = [date for date in possible_dates if date.split()[0] not in unique_months]

    # Step 3: Filter out remaining dates based on the logic of the puzzle
    for date in possible_dates:
        month, day = date.split()
        if all(d.split()[0] != month for d in possible_dates if d != date):
            return date

# List of possible dates
possible_dates = [
    "May 15", "May 16", "May 19",
    "June 17", "June 18",
    "July 14", "July 16",
    "August 14", "August 15", "August 17"
]

birthday = cheryls_birthday(possible_dates)
print(f"Cheryl's birthday is: {birthday}")

Cheryl's birthday is: None


## Gemini Advanced

*[Gemini makes a similar mistake.]*


In [3]:
def cheryl_birthday(possible_dates):
    """Solves the Cheryl's Birthday logic problem, allowing for a variable list of possible dates."""

    # Possible months
    months = ["May", "June", "July", "August"]

    # Albert's statement eliminates some possibilities
    # He knows the month, so there must be only one possible date in that month
    for month in months:
        if len([date for date in possible_dates if (month, date) not in [("May", 19), ("June", 18)]]) == 1:
            months = [month]
            break

    # Bernard's statement further narrows down the possibilities
    # He knows the date, so there must be only one possible month for that date
    for date in possible_dates:
        if len([month for month in months if (month, date) not in [("May", 19), ("June", 18)]]) == 1:
            possible_dates = [date]
            break

    # After Bernard's statement, Albert also knows the answer
    # This means there's only one possible date left in the remaining month
    for month in months:
        if len([date for date in possible_dates if (month, date) not in [("May", 19), ("June", 18)]]) == 1:
            possible_dates = [date]
            break

    # Now there should be only one possibility left
    if len(months) == 1 and len(possible_dates) == 1:
        return months[0], possible_dates[0]
    else:
        return None

# Original problem setup
original_dates = [14, 15, 16, 17, 18, 19]
solution = cheryl_birthday(original_dates)

if solution:
    print(f"Cheryl's birthday is on {solution[0]} {solution[1]}")
else:
    print("Unable to determine Cheryl's birthday")

# Example with a different set of possible dates
new_dates = [13, 14, 15, 16, 17]
new_solution = cheryl_birthday(new_dates)

if new_solution:
    print(f"With the new set of dates, Cheryl's birthday is on {new_solution[0]} {new_solution[1]}")
else:
    print("With the new set of dates, unable to determine Cheryl's birthday")

Unable to determine Cheryl's birthday
With the new set of dates, unable to determine Cheryl's birthday


*[The program accepts a list of "possible_dates", but confusingly, those are actually just days of the month; you aren't allowed to pass in different month/day combinations. And even if you do pass in different days, the program hard-codes certain days and months.]*

## Meta Llama 405B

*[Llama also fails to find a solution (and thus has no output). This program is quite flawed. I do like that it defines functions for the various pieces, but note that the function "bernard_deduce" is defined but never called.]*

In [4]:
# Possible dates for Cheryl's birthday
dates = [
    (5, 13), (5, 15), (5, 16), 
    (6, 13), (6, 15), (6, 16), 
    (7, 13), (7, 15), (7, 16), 
    (8, 13), (8, 15), (8, 16)
]

# Function to check if a date is possible based on Albert's knowledge
def is_possible_albert(month):
    return all(m == month for m, _ in dates)

# Function to check if a date is possible based on Bernard's knowledge
def is_possible_bernard(day):
    return any((m, d) == (month, day) for m, d in dates)

# Function to check if Albert can deduce the birthday
def albert_deduce(month):
    return len([d for m, d in dates if m == month]) == 1

# Function to check if Bernard can deduce the birthday
def bernard_deduce(day):
    return len([m for m, d in dates if d == day]) == 1

# Main program
for month, day in dates:
    if not is_possible_albert(month):
        continue
    if not is_possible_bernard(day):
        continue
    if albert_deduce(month):
        print(f"Cheryl's birthday is {month}/{day}")
        break

## Claude 3.5 Sonnet

*[Claude makes a similar mistake. It deserves credit for explicitly trying an example with different dates, and it correctly reports that the second example has no solution, but that was by accident.]*

In [5]:
def solve_cheryls_birthday(dates):
    # Extract unique months and days
    months = sorted(set(date.split()[0] for date in dates))
    days = sorted(set(int(date.split()[1]) for date in dates))

    # Step 1: Albert knows the month, Bernard knows the day
    possible_months = [m for m in months if len([d for d in dates if d.startswith(m)]) > 1]
    possible_days = [d for d in days if len([date for date in dates if date.endswith(str(d))]) > 1]

    # Step 2: Albert knows Bernard doesn't know
    possible_dates = [date for date in dates if int(date.split()[1]) in possible_days]

    # Step 3: Bernard now knows
    possible_dates = [date for date in possible_dates if possible_dates.count(date.split()[1]) == 1]

    # Step 4: Albert now knows
    result = [date for date in possible_dates if possible_dates.count(date.split()[0]) == 1]

    return result[0] if result else "No unique solution found."

# Example usage
dates = [
    "May 15", "May 16", "May 19",
    "June 17", "June 18",
    "July 14", "July 16",
    "August 14", "August 15", "August 17"
]

solution = solve_cheryls_birthday(dates)
print(f"Cheryl's birthday is on: {solution}")

# Test with a different set of dates
new_dates = [
    "April 15", "April 16", "April 25",
    "May 15", "May 16",
    "June 17", "June 18",
    "July 14", "July 16", "July 17"
]

new_solution = solve_cheryls_birthday(new_dates)
print(f"With the new set of dates, Cheryl's birthday is on: {new_solution}")

Cheryl's birthday is on: No unique solution found.
With the new set of dates, Cheryl's birthday is on: No unique solution found.


## Perplexity

*[Once again, this program has a similar mistake.]*

In [6]:
from collections import defaultdict

def solve_cheryls_birthday(possible_dates):
    months = defaultdict(list)
    days = defaultdict(list)
    
    for date in possible_dates:
        month, day = date.split()
        months[month].append(date)
        days[day].append(date)
    
    # Albert knows the month, Bernard knows the day
    possible = [date for date in possible_dates if len(months[date.split()[0]]) > 1 and len(days[date.split()[1]]) > 1]
    
    # Bernard now knows the date
    possible = [date for date in possible if len([d for d in possible if d.split()[1] == date.split()[1]]) == 1]
    
    # Albert now knows the date
    possible = [date for date in possible if len([d for d in possible if d.split()[0] == date.split()[0]]) == 1]
    
    return possible[0] if len(possible) == 1 else None

# Example usage
possible_dates = [
    "May 15", "May 16", "May 19",
    "June 17", "June 18",
    "July 14", "July 16",
    "August 14", "August 15", "August 17"
]

result = solve_cheryls_birthday(possible_dates)
print(f"Cheryl's birthday is: {result}")

Cheryl's birthday is: None


## Cohere Chat Playground

*[Cohere too has the same mistake.]*

In [7]:
def solve_cheryls_birthday(possible_dates):
    # Create a dictionary to map months to their respective days
    month_days = {
        "May": [15, 16, 19],
        "June": [17, 18],
        "July": [14, 16],
        "August": [14, 15, 17],
    }

    # Albert knows the month, so we can filter out the possible dates based on that
    albert_knows = [date for date in possible_dates if date[1] in month_days]

    # Bernard knows the day, so we can further filter based on that
    bernard_knows = [date for date in albert_knows if date[2] in month_days[date[1]]]

    # If Bernard knows the birthday, Albert should also know it
    if len(bernard_knows) == 1:
        return bernard_knows[0]
    else:
        return "The solution could not be determined."

# Example usage:
possible_dates = [
    ("May", 15, "Friday"),
    ("May", 16, "Saturday"),
    ("May", 19, "Tuesday"),
    ("June", 17, "Thursday"),
    ("June", 18, "Friday"),
    ("July", 14, "Monday"),
    ("July", 16, "Wednesday"),
    ("August", 14, "Wednesday"),
    ("August", 15, "Thursday"),
    ("August", 17, "Saturday"),
]

solution = solve_cheryls_birthday(possible_dates)
print(f"Cheryl's birthday is: {solution}")

Cheryl's birthday is: The solution could not be determined.


*[This program seems promising in that it has separate variables for "albert_knows" and "bernard_knows", but it does not handle them  correctly. The program introduces days of the week, which is extraneous, but not harmful. A bigger issue is that it accepts "possible_dates" as a parameter, but then ignores them and defines "month_days" based on the original dates, not on the input.]*

## HuggingFace Chat

*[HuggingFace is unique in suggesting a class definition, but still makes a similar mistake. Also, while it accepts a list of possible dates in the initializer, it then hard-codes certain months and days from the original problem.]*

In [8]:
class CherylBirthdayProblem:
    def __init__(self, possible_dates):
        self.possible_dates = possible_dates    
        
    def filter_dates(self, month, day):
        return [date for date in self.possible_dates if date[0] == month and date[1] == day]
    
    def filter_months(self, month):
        return [date for date in self.possible_dates if date[0] == month]
    
    def filter_days(self, day):
        return [date for date in self.possible_dates if date[1] == day]
    
    def solve(self):
        # Albert's first statement
        self.possible_dates = self.filter_months(5) + self.filter_months(6)
    
        # Bernard's first statement
        self.possible_dates = [date for date in self.possible_dates if date[1]!= 19]
    
        # Albert's second statement
        self.possible_dates = self.filter_days(14) + self.filter_days(15) + self.filter_days(16) + self.filter_days(17) + self.filter_days(18)
    
        # Bernard's second statement
        self.possible_dates = [date for date in self.possible_dates if date[1]!= 15 and date[0]!= 6]
    
        # Albert's third statement
        self.possible_dates = self.filter_months(7)
    
        # Bernard's third statement
        self.possible_dates = self.filter_days(16)
    
        return self.possible_dates

possible_dates = [(5, 14), (5, 15), (5, 16), (6, 15), (6, 16), (6, 17), (6, 18), (6, 19), 
                  (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (8, 14), (8, 15), (8, 16), 
                  (8, 17), (8, 18), (8, 19)]
problem = CherylBirthdayProblem(possible_dates)
print(problem.solve())

[]


## You.com Genius Mode

*[You.com makes a similar mistake, although the end result is that it selects the wrong date rather than no date.]*

In [9]:
def cheryls_birthday(possible_dates):
    # Step 1: Albert knows Bernard doesn't know. So, the month can't be one with a unique day.
    step1 = [date for date in possible_dates 
             if not any(possible_dates.count(x) == 1 
                        for x in [d for m, d in possible_dates if m == date[0]])]

    # Step 2: Bernard now knows. So, the day must be unique among the remaining possible dates.
    step2 = [date for date in step1 if sum(date[1] == d for m, d in step1) == 1]

    # Step 3: Albert now knows. So, the month must be unique among the remaining possible dates.
    step3 = [date for date in step2 if sum(date[0] == m for m, d in step2) == 1]

    return step3[0] if step3 else None

# List of possible dates
possible_dates = [('May', 15), ('May', 16), ('May', 19), ('June', 17), ('June', 18), 
                  ('July', 14), ('July', 16), ('August', 14), ('August', 15), ('August', 17)]

print(cheryls_birthday(possible_dates))

('May', 19)


# 2025: Programming

In 2025, three of the LLMs solved the programming test. I'll show Gemini and Claude. I'll test them on the original dates (which each program defines) and on the following three sets of alternative dates, two that have a unique solution, and one with no solution.

In [10]:
alt_dates = [('January', 15), ('January', 4), ('July', 13), ('July', 24), ('July', 30), 
             ('March', 13), ('March', 24), ('May', 11), ('May', 17), ('May', 30)] # Answer: July 30

alt_dates2 = [('April', 17), ('April', 18), ('April', 28), ('July', 16), ('July', 17), 
              ('July', 19), ('June', 16), ('June', 29), ('March', 18), ('March', 19)] # Answer: March 18

bad_dates = [('January', 15), ('January', 4), ('July', 13), ('July', 24), ('July', 30), 
             ('March', 13), ('March', 24), ('May', 11), ('May', 17), ('May', 13)] # Answer: None

## Gemini Pro 2.5

Here's the program from **Gemini Pro 2.5** in 2025. It passes all the tests. However, there is a serious flaw: some of the explanatory output that is produces consists of static strings hard-coded to the original set of dates. The program still gets the right solution when given an alternative set of dates as input, but the explanatory output is wrong. (Also, I would have provided a keyword argument to make the printing optional.) I have to say that overall I like the style of [my program](Cheryl.ipynb) better, but this program certainly does the job (other than the issue with the explanatory output). Gemini's "Show Thinking" description of the program (not shown here) was also cogent, helpful, and correct.

In [11]:
from collections import Counter

def print_dates(dates, step_message):
    """Helper function to print the current set of possible dates."""
    print(f"\n## {step_message}")
    if not dates:
        print("No possible dates remain.")
        return
    months = sorted(list(set(d[0] for d in dates)))
    print("Possible dates:")
    for month in months:
        days = sorted([d[1] for d in dates if d[0] == month])
        print(f"- {month}: {', '.join(map(str, days))}")
    print(f"({len(dates)} possibilities)")

def solve_cheryl_birthday(possible_dates):
    """
    Solves Cheryl's Birthday puzzle based on the statements.

    Args:
        possible_dates: A list of tuples, where each tuple is (month, day).
    """

    print("--- Starting Cheryl's Birthday Puzzle Solver ---")
    print_dates(possible_dates, "Initial Possible Dates")

    # --- Statement 1: Albert: "I don't know when Cheryl's birthday is,
    #                  and I know that Bernard does not know." ---

    # Part 1.1: Bernard does not know.
    # If a day is unique in the initial list, Bernard *would* know if he
    # was told that day. Albert *knows* Bernard doesn't know. This means
    # Albert's month *cannot* contain any day that is initially unique.
    # So, we first find the days that appear only once (unique days).
    initial_day_counts = Counter(d[1] for d in possible_dates)
    unique_days = {day for day, count in initial_day_counts.items() if count == 1}

    # Now, we filter out any date whose day is unique. If a month contains
    # *any* unique day, Albert couldn't be sure Bernard didn't know.
    # So, we must eliminate any month that has a date with a unique day.
    months_with_unique_days = {d[0] for d in possible_dates if d[1] in unique_days}
    dates_after_albert1a = [d for d in possible_dates if d[0] not in months_with_unique_days]

    print_dates(dates_after_albert1a, "After Albert knows Bernard doesn't know")
    print("  *Logic: We removed months (like May and June) that contained 'unique' days (18 or 19).")
    print("   If Albert had May or June, he couldn't be certain Bernard didn't know.*")

    # Part 1.2: Albert does not know.
    # This means that *after* the previous filtering (or even before),
    # Albert's month must still have more than one possible date.
    # We check the months remaining in 'dates_after_albert1a'. If any of
    # these months now has only one date, it couldn't be Albert's month.
    # However, in this specific problem, this step doesn't eliminate more dates,
    # as both July and August still have multiple options. We'll keep the dates
    # from the previous step.
    dates_after_albert1 = dates_after_albert1a
    # Optional: Add a check here if needed for other datasets.
    # month_counts_a1 = Counter(d[0] for d in dates_after_albert1a)
    # dates_after_albert1 = [d for d in dates_after_albert1a if month_counts_a1[d[0]] > 1]
    # In this case, it yields the same result.

    print_dates(dates_after_albert1, "After Albert says 'I don't know'")
    print("  *Logic: Albert's month must still have multiple options. (In this case, no dates were removed)*")


    # --- Statement 2: Bernard: "At first I didn't know..., but I know now." ---
    # Bernard heard Albert's statement, so he knows the list is now
    # 'dates_after_albert1'. Since he *now* knows, it means his day
    # must be unique within *this new list*.
    day_counts_b1 = Counter(d[1] for d in dates_after_albert1)
    dates_after_bernard1 = [d for d in dates_after_albert1 if day_counts_b1[d[1]] == 1]

    print_dates(dates_after_bernard1, "After Bernard says 'I know now'")
    print("  *Logic: We looked at the current list (July & August dates). We kept only those dates")
    print("   whose day appears exactly once in this list (16, 15, 17). We removed 14 because")
    print("   it appeared twice, meaning Bernard wouldn't know if his day was 14.*")

    # --- Statement 3: Albert: "Then I also know when Cheryl's birthday is." ---
    # Albert heard Bernard's statement, so he knows the list is now
    # 'dates_after_bernard1'. Since Albert *now* knows, it means his
    # month must be unique within *this new list*.
    month_counts_a2 = Counter(d[0] for d in dates_after_bernard1)
    final_dates = [d for d in dates_after_bernard1 if month_counts_a2[d[0]] == 1]

    print_dates(final_dates, "After Albert says 'Then I also know'")
    print("  *Logic: We looked at the current list (July 16, Aug 15, Aug 17). We kept only those dates")
    print("   whose month appears exactly once. August appears twice, so if Albert had August,")
    print("   he wouldn't know. Since he knows, he must have July.*")

    # --- The Solution ---
    print("\n" + "="*40)
    if len(final_dates) == 1:
        print(f"🎉 The solution is: {final_dates[0][0]} {final_dates[0][1]} 🎉")
    elif len(final_dates) > 1:
        print("🤔 The statements lead to multiple possibilities.")
    else:
        print("😥 No solution found based on the statements.")
    print("="*40)

    return final_dates

# --- Run the puzzle with the original dates ---
if __name__ == "__main__":
    cheryl_dates = [
        ('May', 15), ('May', 16), ('May', 19),
        ('June', 17), ('June', 18),
        ('July', 14), ('July', 16),
        ('August', 14), ('August', 15), ('August', 17)
    ]

    solve_cheryl_birthday(cheryl_dates)

--- Starting Cheryl's Birthday Puzzle Solver ---

## Initial Possible Dates
Possible dates:
- August: 14, 15, 17
- July: 14, 16
- June: 17, 18
- May: 15, 16, 19
(10 possibilities)

## After Albert knows Bernard doesn't know
Possible dates:
- August: 14, 15, 17
- July: 14, 16
(5 possibilities)
  *Logic: We removed months (like May and June) that contained 'unique' days (18 or 19).
   If Albert had May or June, he couldn't be certain Bernard didn't know.*

## After Albert says 'I don't know'
Possible dates:
- August: 14, 15, 17
- July: 14, 16
(5 possibilities)
  *Logic: Albert's month must still have multiple options. (In this case, no dates were removed)*

## After Bernard says 'I know now'
Possible dates:
- August: 15, 17
- July: 16
(3 possibilities)
  *Logic: We looked at the current list (July & August dates). We kept only those dates
   whose day appears exactly once in this list (16, 15, 17). We removed 14 because
   it appeared twice, meaning Bernard wouldn't know if his day was 1

Let's verify that it computes the correct solution with the alternative sets of dates:

In [12]:
assert solve_cheryl_birthday(alt_dates) == [('July', 30)]

--- Starting Cheryl's Birthday Puzzle Solver ---

## Initial Possible Dates
Possible dates:
- January: 4, 15
- July: 13, 24, 30
- March: 13, 24
- May: 11, 17, 30
(10 possibilities)

## After Albert knows Bernard doesn't know
Possible dates:
- July: 13, 24, 30
- March: 13, 24
(5 possibilities)
  *Logic: We removed months (like May and June) that contained 'unique' days (18 or 19).
   If Albert had May or June, he couldn't be certain Bernard didn't know.*

## After Albert says 'I don't know'
Possible dates:
- July: 13, 24, 30
- March: 13, 24
(5 possibilities)
  *Logic: Albert's month must still have multiple options. (In this case, no dates were removed)*

## After Bernard says 'I know now'
Possible dates:
- July: 30
(1 possibilities)
  *Logic: We looked at the current list (July & August dates). We kept only those dates
   whose day appears exactly once in this list (16, 15, 17). We removed 14 because
   it appeared twice, meaning Bernard wouldn't know if his day was 14.*

## After Albe

In [13]:
assert solve_cheryl_birthday(alt_dates2) == [('March', 18)]

--- Starting Cheryl's Birthday Puzzle Solver ---

## Initial Possible Dates
Possible dates:
- April: 17, 18, 28
- July: 16, 17, 19
- June: 16, 29
- March: 18, 19
(10 possibilities)

## After Albert knows Bernard doesn't know
Possible dates:
- July: 16, 17, 19
- March: 18, 19
(5 possibilities)
  *Logic: We removed months (like May and June) that contained 'unique' days (18 or 19).
   If Albert had May or June, he couldn't be certain Bernard didn't know.*

## After Albert says 'I don't know'
Possible dates:
- July: 16, 17, 19
- March: 18, 19
(5 possibilities)
  *Logic: Albert's month must still have multiple options. (In this case, no dates were removed)*

## After Bernard says 'I know now'
Possible dates:
- July: 16, 17
- March: 18
(3 possibilities)
  *Logic: We looked at the current list (July & August dates). We kept only those dates
   whose day appears exactly once in this list (16, 15, 17). We removed 14 because
   it appeared twice, meaning Bernard wouldn't know if his day was 14.

In [14]:
assert solve_cheryl_birthday(bad_dates) == []

--- Starting Cheryl's Birthday Puzzle Solver ---

## Initial Possible Dates
Possible dates:
- January: 4, 15
- July: 13, 24, 30
- March: 13, 24
- May: 11, 13, 17
(10 possibilities)

## After Albert knows Bernard doesn't know
Possible dates:
- March: 13, 24
(2 possibilities)
  *Logic: We removed months (like May and June) that contained 'unique' days (18 or 19).
   If Albert had May or June, he couldn't be certain Bernard didn't know.*

## After Albert says 'I don't know'
Possible dates:
- March: 13, 24
(2 possibilities)
  *Logic: Albert's month must still have multiple options. (In this case, no dates were removed)*

## After Bernard says 'I know now'
Possible dates:
- March: 13, 24
(2 possibilities)
  *Logic: We looked at the current list (July & August dates). We kept only those dates
   whose day appears exactly once in this list (16, 15, 17). We removed 14 because
   it appeared twice, meaning Bernard wouldn't know if his day was 14.*

## After Albert says 'Then I also know'
No pos

## Claude Sonnet 4

Now for the program from **Anthropic Claude Sonnet 4** in 2025. It is similar to the Gemini program. It too produces explanatory output (that can't be turned off). With Claude, the explanatory output is not hard-coded to the original dates, so overall it is a better program.

In [15]:
def solve_cheryls_birthday(possible_dates):
    """
    Solve Cheryl's Birthday problem for any list of possible dates.
    
    Args:
        possible_dates: List of tuples (month, day) representing possible birthdays
    
    Returns:
        The unique solution as a tuple (month, day), or None if no unique solution
    """
    
    print(f"Possible dates: {possible_dates}")
    print()
    
    # Step 1: Albert says "I don't know when Cheryl's birthday is, 
    # but I know that Bernard doesn't know either."
    
    # For Albert to know that Bernard doesn't know, Albert's month must not 
    # contain any unique days (days that appear only once across all dates)
    
    # Find all unique days (days that appear only once)
    day_counts = {}
    for month, day in possible_dates:
        day_counts[day] = day_counts.get(day, 0) + 1
    
    unique_days = {day for day, count in day_counts.items() if count == 1}
    print(f"Unique days (appear only once): {sorted(unique_days)}")
    
    # Find months that contain unique days
    months_with_unique_days = set()
    for month, day in possible_dates:
        if day in unique_days:
            months_with_unique_days.add(month)
    
    print(f"Months containing unique days: {sorted(months_with_unique_days)}")
    
    # Albert's month cannot be one that contains unique days
    # (otherwise he couldn't be sure Bernard doesn't know)
    valid_months = set()
    for month, day in possible_dates:
        if month not in months_with_unique_days:
            valid_months.add(month)
    
    print(f"Valid months for Albert (don't contain unique days): {sorted(valid_months)}")
    
    # Filter possible dates to only those in valid months
    after_step1 = [(month, day) for month, day in possible_dates 
                   if month in valid_months]
    
    print(f"Remaining dates after Albert's statement: {after_step1}")
    print()
    
    # Step 2: Bernard says "At first I didn't know when Cheryl's birthday is, 
    # but I know now."
    
    # For Bernard to know now, his day must appear only once in the remaining dates
    remaining_day_counts = {}
    for month, day in after_step1:
        remaining_day_counts[day] = remaining_day_counts.get(day, 0) + 1
    
    unique_remaining_days = {day for day, count in remaining_day_counts.items() 
                           if count == 1}
    
    print(f"Days that appear only once in remaining dates: {sorted(unique_remaining_days)}")
    
    # Filter to dates where the day is unique in remaining set
    after_step2 = [(month, day) for month, day in after_step1 
                   if day in unique_remaining_days]
    
    print(f"Remaining dates after Bernard's statement: {after_step2}")
    print()
    
    # Step 3: Albert says "Then I also know when Cheryl's birthday is."
    
    # For Albert to know, there must be only one date left with his month
    month_counts = {}
    for month, day in after_step2:
        month_counts[month] = month_counts.get(month, 0) + 1
    
    # Find months that appear only once in remaining dates
    unique_remaining_months = {month for month, count in month_counts.items() if count == 1}
    
    print(f"Months that appear only once in remaining dates: {sorted(unique_remaining_months)}")
    
    # Final answer: dates where the month is unique in remaining set
    final_dates = [(month, day) for month, day in after_step2 
                   if month in unique_remaining_months]
    
    print(f"Final answer: {final_dates}")
    
    if len(final_dates) == 1:
        return final_dates[0]
    else:
        return None

# Original Cheryl's Birthday dates
original_dates = [
    ("May", 15), ("May", 16), ("May", 19),
    ("June", 17), ("June", 18),
    ("July", 14), ("July", 16),
    ("August", 14), ("August", 15), ("August", 17)
]

print("=== SOLVING ORIGINAL CHERYL'S BIRTHDAY PROBLEM ===")
solution = solve_cheryls_birthday(original_dates)
if solution:
    print(f"\n🎉 Cheryl's birthday is: {solution[0]} {solution[1]}")
else:
    print("\n❌ No unique solution found")

=== SOLVING ORIGINAL CHERYL'S BIRTHDAY PROBLEM ===
Possible dates: [('May', 15), ('May', 16), ('May', 19), ('June', 17), ('June', 18), ('July', 14), ('July', 16), ('August', 14), ('August', 15), ('August', 17)]

Unique days (appear only once): [18, 19]
Months containing unique days: ['June', 'May']
Valid months for Albert (don't contain unique days): ['August', 'July']
Remaining dates after Albert's statement: [('July', 14), ('July', 16), ('August', 14), ('August', 15), ('August', 17)]

Days that appear only once in remaining dates: [15, 16, 17]
Remaining dates after Bernard's statement: [('July', 16), ('August', 15), ('August', 17)]

Months that appear only once in remaining dates: ['July']
Final answer: [('July', 16)]

🎉 Cheryl's birthday is: July 16


Here we verify that it works on the alternative dates:

In [16]:
assert solve_cheryls_birthday(alt_dates) == ('July', 30)

Possible dates: [('January', 15), ('January', 4), ('July', 13), ('July', 24), ('July', 30), ('March', 13), ('March', 24), ('May', 11), ('May', 17), ('May', 30)]

Unique days (appear only once): [4, 11, 15, 17]
Months containing unique days: ['January', 'May']
Valid months for Albert (don't contain unique days): ['July', 'March']
Remaining dates after Albert's statement: [('July', 13), ('July', 24), ('July', 30), ('March', 13), ('March', 24)]

Days that appear only once in remaining dates: [30]
Remaining dates after Bernard's statement: [('July', 30)]

Months that appear only once in remaining dates: ['July']
Final answer: [('July', 30)]


In [17]:
assert solve_cheryls_birthday(alt_dates2) == ('March', 18)

Possible dates: [('April', 17), ('April', 18), ('April', 28), ('July', 16), ('July', 17), ('July', 19), ('June', 16), ('June', 29), ('March', 18), ('March', 19)]

Unique days (appear only once): [28, 29]
Months containing unique days: ['April', 'June']
Valid months for Albert (don't contain unique days): ['July', 'March']
Remaining dates after Albert's statement: [('July', 16), ('July', 17), ('July', 19), ('March', 18), ('March', 19)]

Days that appear only once in remaining dates: [16, 17, 18]
Remaining dates after Bernard's statement: [('July', 16), ('July', 17), ('March', 18)]

Months that appear only once in remaining dates: ['March']
Final answer: [('March', 18)]


In [18]:
assert solve_cheryls_birthday(bad_dates) == None

Possible dates: [('January', 15), ('January', 4), ('July', 13), ('July', 24), ('July', 30), ('March', 13), ('March', 24), ('May', 11), ('May', 17), ('May', 13)]

Unique days (appear only once): [4, 11, 15, 17, 30]
Months containing unique days: ['January', 'July', 'May']
Valid months for Albert (don't contain unique days): ['March']
Remaining dates after Albert's statement: [('March', 13), ('March', 24)]

Days that appear only once in remaining dates: [13, 24]
Remaining dates after Bernard's statement: [('March', 13), ('March', 24)]

Months that appear only once in remaining dates: []
Final answer: []
